# A2C (Advantage Actor-Critic) 튜토리얼

이 노트북에서는 A2C (Advantage Actor-Critic) 알고리즘의 핵심 개념을 단계별로 학습합니다.

## 목차
1. Actor-Critic의 등장 배경
2. 배우(Actor)와 평론가(Critic)의 역할
3. 어드밴티지 함수 (Advantage Function)
4. A2C 신경망 아키텍처
5. Actor와 Critic의 손실 함수
6. 전체 학습 루프

---
## 1. Actor-Critic의 등장 배경

강화학습에는 크게 두 가지 접근법이 있습니다:

### 정책 기반 방법 (Policy-based methods, e.g., REINFORCE)

**장점:**
- 연속적인 행동 공간에서도 작동
- 확률적 정책 학습 가능

**단점:**
- **높은 분산(High Variance)**: 같은 상태에서도 매번 다른 보상을 받을 수 있어 학습이 불안정
- 샘플 효율성이 낮음

```python
# REINFORCE의 정책 gradient
# J(θ) = E[Σ log π(a|s) * R]  # R은 총 보상 (분산이 매우 큼!)
```

### 가치 기반 방법 (Value-based methods, e.g., DQN)

**장점:**
- 분산이 낮아 안정적 학습
- 샘플 효율성이 높음

**단점:**
- **이산적 행동 공간만 가능**: 연속적인 행동을 다루기 어려움
- 확률적 정책을 학습할 수 없음 (deterministic)

### 🎯 Actor-Critic의 해결책

Actor-Critic은 **두 방법의 장점을 결합**합니다:
- **Actor**: 정책 기반 (연속 행동 가능)
- **Critic**: 가치 기반 (분산 감소)

→ **Critic의 가치 함수를 baseline으로 사용하여 분산을 줄이면서도 정책을 직접 학습!**

---
## 2. 배우(Actor)와 평론가(Critic)의 역할

### 🎭 Actor (배우)

**역할**: "어떤 행동을 할지" 결정하는 정책 학습

```python
# Actor는 정책 π(a|s)를 출력
action_probs = actor(state)  # [0.1, 0.3, 0.6] - 각 행동의 확률
action = sample(action_probs)  # 확률에 따라 행동 선택
```

**수식**: π(a|s; θ) - 상태 s에서 행동 a를 선택할 확률

### 👨‍⚖️ Critic (평론가)

**역할**: "현재 상태가 얼마나 좋은지" 평가하는 가치 함수 학습

```python
# Critic은 상태 가치 V(s)를 출력
state_value = critic(state)  # 15.3 - 이 상태의 예상 총 보상
```

**수식**: V(s; w) - 상태 s의 가치 (기대 누적 보상)

### 🤝 둘의 상호작용

**비유로 이해하기:**

마리오 게임을 배우는 상황을 상상해봅시다.

- **Actor (배우)**: "여기서 점프해야겠어!" → 행동 실행
- **Critic (평론가)**: "음... 이 위치는 위험해 보이는데? (낮은 점수)" → 평가
- **Actor**: "아, 평론가가 낮게 평가했네. 다음엔 다른 행동을 시도해야겠다" → 학습

→ Critic의 피드백으로 Actor가 더 나은 행동을 학습!

---
## 3. 어드밴티지 함수 (Advantage Function)

### 🤔 문제: 절대적 가치 vs 상대적 가치

Q-value가 높다고 해서 그 행동이 "좋은" 행동일까요?

**예시:**
```
상태 A:
  - 왼쪽으로 가기: Q(s, left) = 10
  - 오른쪽으로 가기: Q(s, right) = 15
  - V(s) = 12.5 (평균)

→ 오른쪽이 "절대적으로" 좋다? (15점)
→ 아니면 "평균보다" 2.5점 더 좋다?
```

### 💡 Advantage Function의 아이디어

**"평균보다 얼마나 더 좋은가?"**를 측정합니다.

```
A(s, a) = Q(s, a) - V(s)
```

**의미:**
- A(s, a) > 0: 평균보다 좋은 행동 → 더 자주 선택하도록 학습
- A(s, a) < 0: 평균보다 나쁜 행동 → 덜 선택하도록 학습
- A(s, a) = 0: 평균적인 행동

### 🎯 분산 감소 원리

**REINFORCE (baseline 없음):**
```python
# 총 보상 R은 매우 불안정 (분산 큼)
policy_gradient = log(π(a|s)) * R
```

**Actor-Critic (baseline 사용):**
```python
# V(s)를 빼서 분산 감소!
policy_gradient = log(π(a|s)) * (R - V(s))
policy_gradient = log(π(a|s)) * A(s, a)  # Advantage!
```

### 📊 실제 계산 방법

Q(s,a)를 직접 계산하는 대신, **TD error**를 사용합니다:

```python
# TD error = 실제 받은 보상 + 다음 상태 가치 - 현재 상태 가치
td_error = reward + γ * V(s') - V(s)

# TD error가 Advantage의 추정치!
A(s, a) ≈ td_error
```

---
## 4. A2C 신경망 아키텍처 (PyTorch)

### 🏗️ 전체 구조

```
입력 (게임 화면)
    ↓
공통 CNN (Feature Extractor)
    ↓
LSTM (시간적 패턴 학습)
    ↓
  ┌─────────┐
  │         │
Actor     Critic
(정책)    (가치)
```

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ActorCritic(nn.Module):
    """
    A2C 신경망: 공통 백본 + Actor/Critic 헤드
    """
    def __init__(self, num_inputs, num_actions):
        super(ActorCritic, self).__init__()
        
        # === 공통 Feature Extractor (CNN) ===
        # 게임 화면에서 중요한 특징을 추출
        self.conv1 = nn.Conv2d(num_inputs, 32, 3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(32, 32, 3, stride=2, padding=1)
        self.conv4 = nn.Conv2d(32, 32, 3, stride=2, padding=1)
        
        # === LSTM (시간적 패턴 학습) ===
        # "이전에 어떤 상황이었는지" 기억
        self.lstm = nn.LSTMCell(32 * 6 * 6, 512)
        
        # === Actor Head (정책 네트워크) ===
        # 출력: 각 행동의 점수 (logits)
        self.actor_linear = nn.Linear(512, num_actions)
        
        # === Critic Head (가치 네트워크) ===
        # 출력: 현재 상태의 가치 (scalar)
        self.critic_linear = nn.Linear(512, 1)
    
    def forward(self, x, hx, cx):
        # CNN으로 특징 추출
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # LSTM 통과
        hx, cx = self.lstm(x, (hx, cx))
        
        # Actor와 Critic 출력
        actor_logits = self.actor_linear(hx)    # [batch, num_actions]
        critic_value = self.critic_linear(hx)   # [batch, 1]
        
        return actor_logits, critic_value, hx, cx

### 🔍 각 레이어의 역할

#### 1. CNN Layers (conv1~4)
- **입력**: 게임 화면 (84x84 grayscale)
- **역할**: 적, 블록, 마리오 위치 등 중요한 시각적 패턴 학습
- **출력**: 고수준 특징 벡터

#### 2. LSTM Layer
- **입력**: CNN 특징 벡터
- **역할**: 시간적 패턴 학습 (예: "방금 점프했으니 다음엔 착지할 것")
- **출력**: 시간 정보를 포함한 특징 벡터 (512차원)

#### 3. Actor Head
- **입력**: LSTM 출력 (512차원)
- **출력**: 각 행동의 logits (12개 행동)
- **의미**: "각 행동이 얼마나 좋아 보이는가"

```python
# Softmax를 통해 확률 분포로 변환
logits = actor_head(features)      # [2.1, 0.5, -1.3, ...]
probs = softmax(logits)            # [0.5, 0.2, 0.1, ...] (합=1)
action = sample(probs)             # 확률에 따라 행동 선택
```

#### 4. Critic Head
- **입력**: LSTM 출력 (512차원)
- **출력**: 상태 가치 V(s) (1개 스칼라)
- **의미**: "지금 상황에서 앞으로 얻을 총 보상의 기댓값"

```python
value = critic_head(features)  # 15.3 (이 상태의 예상 총 보상)
```

---
## 5. Actor와 Critic의 손실 함수

### 🎯 Actor Loss (정책 손실)

**목표**: Advantage가 양수인 행동은 더 자주, 음수인 행동은 덜 자주 선택하도록 학습

**수식:**
```
L_actor = -log(π(a|s)) * A(s,a).detach()
```

**직관적 이해:**

```python
# 예시 1: Advantage가 양수 (좋은 행동)
log_prob = -2.0  # log(π(a|s))
advantage = +5.0  # 평균보다 훨씬 좋음!
actor_loss = -(-2.0) * 5.0 = +10.0

# Gradient descent로 loss 감소
# → log_prob 증가 → π(a|s) 증가 → 이 행동을 더 자주 선택!

# 예시 2: Advantage가 음수 (나쁜 행동)
log_prob = -2.0
advantage = -3.0  # 평균보다 나쁨
actor_loss = -(-2.0) * (-3.0) = -6.0

# Gradient descent로 loss 감소
# → log_prob 감소 → π(a|s) 감소 → 이 행동을 덜 선택!
```

**❗ 중요: `.detach()`를 사용하는 이유**

Advantage는 Critic으로부터 계산되지만, Actor 업데이트 시에는 **Critic을 고정**해야 합니다.

```python
# Advantage 계산
advantage = td_error.detach()  # Critic gradient를 차단!

# Actor만 업데이트
actor_loss = -log_prob * advantage
```

### 📊 Critic Loss (가치 손실)

**목표**: 실제 받은 보상과 예측한 가치의 차이를 최소화

**수식:**
```
L_critic = (Return - V(s))²
```

**직관적 이해:**

```python
# 상황: 마리오가 적을 밟고 100점 획득
predicted_value = 50.0    # Critic의 예측
actual_return = 100.0     # 실제로 받은 보상

critic_loss = (100.0 - 50.0)² = 2500.0  # 큰 오차!

# Gradient descent
# → predicted_value를 100.0에 가깝게 조정
# → 다음엔 이런 상황에서 더 정확하게 예측!
```

### 🎲 Entropy Loss (탐험 보너스)

**목표**: 너무 확신 있는 정책을 방지하고 탐험 장려

**수식:**
```
H(π) = -Σ π(a|s) * log(π(a|s))
L_entropy = -β * H(π)  # β는 entropy coefficient
```

**직관적 이해:**

```python
# 나쁜 경우: 너무 확신 있는 정책 (탐험 부족)
probs = [0.99, 0.01, 0.00, 0.00]  # 거의 한 가지 행동만!
entropy = -0.99*log(0.99) - 0.01*log(0.01) = 0.08  # 낮은 엔트로피

# 좋은 경우: 다양한 행동 시도 (탐험 충분)
probs = [0.3, 0.3, 0.2, 0.2]  # 여러 행동을 고려
entropy = -(4개 확률의 합) = 1.28  # 높은 엔트로피

# Entropy를 최대화 (loss는 음수)
# → 다양한 행동을 시도하도록 유도!
```

### 🔀 Total Loss (통합 손실)

```python
total_loss = -actor_loss + critic_loss - β * entropy_loss
```

**각 항의 역할:**
- **-actor_loss**: 좋은 행동을 더 자주 선택 (정책 개선)
- **+critic_loss**: 가치 예측을 정확하게 (안정적 학습)
- **-β * entropy_loss**: 탐험을 장려 (지역 최적화 방지)

**β (beta) 하이퍼파라미터:**
- β가 크면: 더 많은 탐험 (초반에 유리)
- β가 작으면: 더 적극적인 활용 (후반에 유리)

---
## 6. 전체 학습 루프 (Training Loop) 해부

### 📋 학습 과정 개요

```
1. 경험 수집 (Experience Collection)
   ↓
2. Advantage 계산 (Advantage Estimation)
   ↓
3. 손실 계산 (Loss Computation)
   ↓
4. 역전파 (Backpropagation)
   ↓
5. 반복
```

In [ ]:
# === STEP 1: 경험 수집 (n-step 동안) ===

num_steps = 50  # n-step
log_policies = []
values = []
rewards = []
entropies = []

for step in range(num_steps):
    # Forward pass
    logits, value, hx, cx = model(state, hx, cx)
    
    # 정책 확률 계산
    policy = F.softmax(logits, dim=1)        # [0.3, 0.2, 0.5, ...]
    log_policy = F.log_softmax(logits, dim=1)  # [log(0.3), log(0.2), ...]
    
    # Entropy 계산 (탐험 장려)
    entropy = -(policy * log_policy).sum(1, keepdim=True)
    
    # 행동 샘플링
    m = torch.distributions.Categorical(policy)
    action = m.sample().item()  # 확률에 따라 행동 선택
    
    # 환경에서 한 스텝 실행
    next_state, reward, done, info = env.step(action)
    
    # 경험 저장
    values.append(value)
    log_policies.append(log_policy[0, action])  # 선택한 행동의 log prob
    rewards.append(reward)
    entropies.append(entropy)
    
    state = next_state
    
    if done:
        break

In [ ]:
# === STEP 2: Bootstrap Value (에피소드가 끝나지 않았을 때) ===

R = torch.zeros(1, 1)
if not done:
    # 마지막 상태의 가치로 초기화
    _, R, _, _ = model(state, hx, cx)
    # 의미: "앞으로 받을 보상의 예측값"

# === STEP 3: Advantage 계산 (GAE - Generalized Advantage Estimation) ===

gae = torch.zeros(1, 1)  # GAE 누적값
actor_loss = 0
critic_loss = 0
entropy_loss = 0
next_value = R

# 역순으로 계산 (뒤에서부터)
for value, log_policy, reward, entropy in reversed(list(zip(values, log_policies, rewards, entropies))):
    # GAE 계산
    # gae = δ_t + γ*λ*gae_{t+1}
    # δ_t = r_t + γ*V(s_{t+1}) - V(s_t)  (TD error)
    gae = gae * gamma * tau
    gae = gae + reward + gamma * next_value.detach() - value.detach()
    next_value = value
    
    # Actor loss (정책 gradient)
    actor_loss = actor_loss + log_policy * gae  # 주목! gae가 advantage
    
    # Return 계산 (n-step return)
    R = R * gamma + reward
    
    # Critic loss (MSE)
    critic_loss = critic_loss + (R - value) ** 2 / 2
    
    # Entropy loss
    entropy_loss = entropy_loss + entropy

In [ ]:
# === STEP 4: Total Loss 계산 및 역전파 ===

total_loss = -actor_loss + critic_loss - beta * entropy_loss

# Optimizer 초기화
optimizer.zero_grad()

# 역전파
total_loss.backward()

# Gradient clipping (안정적 학습)
torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)

# Parameter 업데이트
optimizer.step()

print(f"Actor Loss: {-actor_loss.item():.4f}")
print(f"Critic Loss: {critic_loss.item():.4f}")
print(f"Entropy: {entropy_loss.item():.4f}")
print(f"Total Loss: {total_loss.item():.4f}")

### 🔄 학습 루프의 핵심 포인트

#### 1. N-step Experience Collection
- 한 번에 여러 스텝의 경험을 모음
- 배치 학습의 효율성

#### 2. GAE (Generalized Advantage Estimation)
```python
# GAE는 여러 시간 스텝의 TD error를 조합
gae = δ_t + γ*λ*δ_{t+1} + (γ*λ)²*δ_{t+2} + ...

# λ=0: 1-step TD (낮은 분산, 높은 편향)
# λ=1: Monte Carlo (높은 분산, 낮은 편향)
# λ=0.95: 좋은 균형점
```

#### 3. Gradient Clipping
```python
# Gradient가 너무 크면 학습이 불안정
# → 일정 크기로 제한
torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
```

#### 4. `.detach()` 사용
```python
# Critic의 gradient가 Actor로 흐르지 않도록 차단
advantage = (reward + gamma * next_value.detach() - value.detach())
```

---
## 🎓 요약 및 핵심 개념

### A2C의 핵심 아이디어

1. **Actor-Critic 분리**
   - Actor: 정책 학습 (어떤 행동을 할지)
   - Critic: 가치 학습 (상황이 얼마나 좋은지)

2. **Advantage Function**
   - A(s,a) = Q(s,a) - V(s)
   - "평균보다 얼마나 좋은가"
   - 분산 감소 효과

3. **통합 손실 함수**
   ```
   L = -L_actor + L_critic - β*L_entropy
   ```
   - 정책 개선 + 가치 예측 + 탐험

4. **안정적 학습을 위한 테크닉**
   - Gradient clipping
   - `.detach()` for advantage
   - GAE for better advantage estimation

### 🚀 다음 스텝

이제 실제로 학습을 실행해봅시다!

```bash
# 학습 시작
python training/train_a2c.py --num-updates 10000

# 학습된 모델 데모
python games/demo_trained_agent.py --model-path models/saved_weights/mario_a3c_XXX.pth
```

### 📚 추가 학습 자료

- [Original A3C Paper](https://arxiv.org/abs/1602.01783)
- [GAE Paper](https://arxiv.org/abs/1506.02438)
- [OpenAI Spinning Up](https://spinningup.openai.com/en/latest/algorithms/vpg.html)

---
## 🧪 실습: 간단한 환경에서 A2C 테스트

CartPole 환경에서 A2C가 어떻게 작동하는지 확인해봅시다.

In [ ]:
# 필요한 라이브러리 임포트
import gymnasium as gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import numpy as np
import matplotlib.pyplot as plt

# 간단한 A2C 모델 (CartPole용)
class SimpleActorCritic(nn.Module):
    def __init__(self, input_dim, num_actions):
        super().__init__()
        self.fc = nn.Linear(input_dim, 128)
        self.actor = nn.Linear(128, num_actions)
        self.critic = nn.Linear(128, 1)
    
    def forward(self, x):
        x = F.relu(self.fc(x))
        return self.actor(x), self.critic(x)

# 환경 생성
env = gym.make('CartPole-v1')
model = SimpleActorCritic(4, 2)  # 상태 4차원, 행동 2개
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print("간단한 A2C 모델 준비 완료!")
print(f"상태 차원: {env.observation_space.shape[0]}")
print(f"행동 개수: {env.action_space.n}")

In [ ]:
# 학습 루프 (간단 버전)
episode_rewards = []
num_episodes = 500

for episode in range(num_episodes):
    state, _ = env.reset()
    done = False
    episode_reward = 0
    
    log_probs = []
    values = []
    rewards = []
    
    # 에피소드 진행
    while not done:
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        logits, value = model(state_tensor)
        
        policy = F.softmax(logits, dim=1)
        m = Categorical(policy)
        action = m.sample()
        
        next_state, reward, terminated, truncated, _ = env.step(action.item())
        done = terminated or truncated
        
        log_probs.append(m.log_prob(action))
        values.append(value)
        rewards.append(reward)
        
        state = next_state
        episode_reward += reward
    
    # 손실 계산 및 업데이트
    R = 0
    returns = []
    for r in reversed(rewards):
        R = r + 0.99 * R
        returns.insert(0, R)
    
    returns = torch.tensor(returns)
    values = torch.cat(values)
    log_probs = torch.stack(log_probs)
    
    advantage = returns - values.detach()
    
    actor_loss = -(log_probs * advantage).mean()
    critic_loss = F.mse_loss(values.squeeze(), returns)
    
    loss = actor_loss + critic_loss
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    episode_rewards.append(episode_reward)
    
    if episode % 50 == 0:
        avg_reward = np.mean(episode_rewards[-50:])
        print(f"Episode {episode}: Avg Reward = {avg_reward:.2f}")

print("학습 완료!")

In [ ]:
# 학습 곡선 시각화
plt.figure(figsize=(12, 4))

# Moving average
window = 50
moving_avg = np.convolve(episode_rewards, np.ones(window)/window, mode='valid')

plt.plot(episode_rewards, alpha=0.3, label='Episode Reward')
plt.plot(moving_avg, label=f'{window}-Episode Moving Average')
plt.xlabel('Episode')
plt.ylabel('Reward')
plt.title('A2C Learning Curve on CartPole')
plt.legend()
plt.grid(True)
plt.show()

print(f"최종 100 에피소드 평균 보상: {np.mean(episode_rewards[-100:]):.2f}")